# Introduction tutorial

This tutorial is an extended version of the quickstart quide in the README and launches the marginalisation script from the Jupyter notebook interface.

We will assume that you have cloned the repo, created the conda environment from the `environment.yml` file and created your `config_local.ini`.

## Input parameters

### Input and output data paths

To be able to do our example run on the provided data, you need to point the local configfile to your local repository clone:
```ini
[data_paths]
local_path = /Users/<YourUser>/repos/ExoTiC-ISM
```

To run the script on the provided example data on the repository, you can keep the default input data path in the configfile, as it points to the data directory in the respository.
```ini
[data_paths]
...
input_path = ${local_path}/data
```

You will need to specifiy a location on disk for the output data:
```ini
[data_paths]
...
output_path = /Users/MyUser/outputs
```

And if you like, you can change the experiment suffix that will be attached to the filename of your output folder:
```ini
[data_paths]
...
run_name = notebook_test
```

In the end the `[data_paths]` section in the configfile should look something like this:
```ini
[data_paths]
local_path = /Users/MyUser/repos/ExoTiC-ISM
input_path = ${local_path}/data
output_path = /Users/MyUser/outputs
run_name = notebook_test
```

### General setup

For the general setup, we are interested in what instrument and grating was used to collect the data on which specific stellar system. This is done in the configfile section `[setup]`.

First, define what star the data is from - the example data is of Wasp-17b, so we dubbed the stellar system section `W17`:
```ini
[setup]
data_set = W17
```

The instrument for this data is Wide Field Camera 3 (WFC3) and the data was taken with the IR grating G141:
```ini
[setup]
...
instrument = WFC3
grating = G141
```

You need to decide which general grid of systematic models you want to use. For WFC3, this grid consists of 50 distinct systematic models used to fit the data, in all of which the transit depth `rl` and the baseline flux `flux0` are always free parameters. The choice you have to make is which of the variable parameters **epoch**, **inclination**, **MsMpR** and **eccentricity** you want to keep fixed ("frozen") or free ("thawed"):

- `fix_time`: all of them are frozen
- `fit_time`: epoch is thawed, other three are frozen
- `fit_inclin`: inclination is thawed, other three are frozen
- `fit_msmpr`: MsMpR is thawed, other three are frozen
- `fit_ecc`: eccentricity is thawed, other three are frozen
- `fit_all`: eccentricity is frozen, other three are thawed

```ini
[setup]
...
grid_selection = fit_time
```

The repository provides a range of both 1D and 3D limb darkening models, so you need to chose which ones to use:
```ini
[setup]
...
ld_model = 3D
```

And finally, you need to decide whether you want the fit results of each systematic model to be displayed at runtime, which can be helpful for troubleshooting but also annoying when trying to work while the code is running, and whether you want the PDF report with the results to be created. It is highly recommended to always keep this `True`, we included the feature to disable this purely to be able to skip this step if the required python packages for this are not available for any reason.

```ini
[setup]
...
plotting = True
report = True
```

In the end, your general setup should look something like this:
```ini
[setup]
data_set = W17
instrument = WFC3
grating = G141
grid_selection = fit_time
ld_model = 3D
plotting = True
report = True
```

### Smooth model parameters

At convenient points in the code, the marginalisation script will calculate a smooth model of the fit. The following configfile section defines at what resultion it gets calculated and what its range in terms of planet phase it will encapsulate. You might need to adjust especially the `half_range` if your data has a significantly different transit length that the example data. For `W17` we use:

```ini
[smooth_model]
resolution = 0.0001
half_range = 0.2
```

### Stellar and planetary system parameters

The last important bit is to define the input parameters for the star and planet. For each new data set you want to analyse, you need to create a new section in your configfile that carries the same section name like what you put in the `data_set` name in `[section] -> data_set`. For the example data, we have a section called `[W17]` that holds all stellar parameters (for limb darkening) and the initial guesses for the fitting parameters for the planetary transit.

```ini
[W17]
...
rl = 0.12169232
epoch = 57957.970153390
inclin = 87.34635
ecc = 0.0
omega = 0.0
Per = 3.73548535
aor = 7.0780354

; limb darkening parameters
metallicity = -1.0
Teff = 6550
logg = 4.5
```

At the top of this section is where you drop in the file names of your data files, one lightcurve file and one file with a wavelength array. If you have more than one data set available for a specific system, your filenames should indicate by what they differ, e.g. different gratings (e.g. G141 or G102), so that you have to make only one single change in your configfile when your changing from one data set to the other. For example, if you have W17 data both with the G141 as well as the G102 grating, you can include the grating name in the filenames and adjust them in the configfile in such a way that the correct files get picked when you adjust your grating of choice in `[setup] -> grating`.

```ini
[setup]
...
grating = G141
...

[W17]
lightcurve_file = W17_${setup:grating}_lightcurve_test_data.txt
wvln_file = W17_${setup:grating}_wavelength_test_data.txt
```

which represents these data files:  
```bash
W17_G141_lightcurve_test_data.txt
W17_G141_wavelength_test_data.txt
```

## Fitting with Sherpa

Before turning to the full marginalisation code, let us have a look at how the fitting is performed. We use a package called `sherpa` (https://sherpa.readthedocs.io/en/latest/). `Sherpa` allows you to write your own model that subsequently gets fit to your data with a **statistic** and an **optimizer** of your choice. ExoTiC-ISM uses the **chi-squared** statistic and the **Levenberg-Marquardt** optimizer, which you can read more about in [Wakeford et al. (2016)](https://ui.adsabs.harvard.edu/abs/2016ApJ...819...10W/abstract) and in the `sherpa` documentation, as well as in any standard sources for least-squares optimisations.

We have transfered our transit model, which is the analytic transit model from [Mandel & Agol (2002)](https://ui.adsabs.harvard.edu/abs/2002ApJ...580L.171M/abstract) into the `sherpa` model format. To make sure this works, we can instantiate a simple transit model based on a simple data set that we will create here.

Just as for a real data set, we created a configfile section for our custom transit, containing the initial guess for the fitting parameters as well as the limb darkening parameters of the star.

```ini
[simple_transit]
rl = 0.1
epoch = 0.
inclin = 90
ecc = 0.0
omega = 0.0
Per = 3.5
aor = 7.0

; limb darkening parameters
metallicity = 0.0
Teff = 5500
logg = 4.5
```

In [ ]:
# First some imports
import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.constants import G

os.chdir('../../exotic-ism')
import margmodule as marg
from config import CONFIG_INI

In [ ]:
# Create simple phase array, grouped in three observation sets
data_x = np.array([-0.046, -0.044, -0.042, -0.040, -0.038, -0.036, -0.034,
                   -0.032, -0.030, -0.006, -0.004, -0.002, 0.0, 0.002, 0.004,
                   0.006, 0.008, 0.01, 0.032, 0.034, 0.036, 0.038, 0.040,
                   0.042, 0.044, 0.046,0.048])

# Create simple light curve
data_y = np.array([1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000,
                   1.0000000, 1.0000000, 1.0000000, 1.0000000, 0.99000000,
                   0.99000000, 0.99000000, 0.99000000, 0.99000000, 0.99000000,
                   0.99000000, 0.99000000, 0.99000000, 1.0000000, 1.0000000,
                   1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000,
                   1.0000000, 1.0000000])

# Assume each data point has exactly the same uncertainty (error)
uncertainty = np.array([0.0004] * len(data_x))

# Add random scatter to the light curve
random_scatter = np.array([0.32558253, -0.55610514, -1.1150768, -1.2337022, -1.2678875,
                           0.60321692, 1.1025507, 1.5080730, 0.76113001, 0.51978011,
                           0.72241364, -0.086782108, -0.22698337, 0.22780245, 0.47119014,
                           -2.1660677, -1.2477670, 0.28568456, 0.40292731, 0.077955817,
                           -1.1090623, 0.66895172, -0.59215439, 0.79973968, 1.0603756,
                           0.82684954, -1.8334587])

data_y += random_scatter * uncertainty

We can have a quick look at our created data with `matplotlib`:

In [ ]:
plt.title('Simple data')
plt.errorbar(data_x, data_y, yerr=uncertainty, fmt='.')
plt.xlabel('Phase')
plt.ylabel('Flux')

To be able to use `sherpa`, we need to create a `sherpa` data object from our simple data, which has its own methods to plot the data. This is exactly the same data we are just showing a different way to plot it. 

In [ ]:
from sherpa.data import Data1D
from sherpa.plot import DataPlot

data = Data1D('simple_transit', data_x, data_y, staterror=uncertainty)   # create data object
dplot = DataPlot()         # create data *plot* object
dplot.prepare(data)       # prepare plot
dplot.plot()

In order to instantiate the transit model, we need to read some of the parameters from the configfile.

In [ ]:
# Simple transit parameters
planet_sys = CONFIG_INI.get('setup', 'data_set')
dtosec = CONFIG_INI.getfloat('constants', 'dtosec')
period = CONFIG_INI.getfloat(planet_sys, 'Per')
Per = period * dtosec
aor = CONFIG_INI.getfloat(planet_sys, 'aor')
constant1 = (G * Per * Per / (4 *np.pi * np.pi))**(1/3)
msmpr = (aor/(constant1))**3

print('msmpr: {}'.format(msmpr))
print('G: {}'.format(G.value))
print('Per: {} sec'.format(Per))

In this example, we will provide limb darkening coefficients that make sense. In the main marginalization script, these get calculated based on the configfile inputs.

In [ ]:
# Limb darkening coefficients
c1 = 0.66396105
c2 = -0.12617095
c3 = 0.053649047
c4 = -0.026713433

Now we can go ahead and instantiate a transit object wit our inputs.

In [ ]:
model = marg.Transit(data_x[0], msmpr, c1, c2, c3, c4, flux0=data_y[0], x_in_phase=True, name='transit_model', sh=None)

We can easily check which fitting parameters are currently thawed and whcih are frozen, and what their initial guesses are.

In [ ]:
print(model)

To keep things simple, we will freeze almost all parameters except for the tranist depth `rl` and the baseline flux `flux0`.

In [ ]:
# Freese almost all parameters
model.epoch.freeze()
model.inclin.freeze()
model.msmpr.freeze()
model.ecc.freeze()
model.m_fac.freeze()
model.hstp1.freeze()
model.hstp2.freeze()
model.hstp3.freeze()
model.hstp4.freeze()
model.xshift1.freeze()
model.xshift2.freeze()
model.xshift3.freeze()
model.xshift4.freeze()

print(model)

In order to visualize the model, we need to calculate it on a grid of x-values. To do so, will will create a smooth and uniform x-array.

In [ ]:
x_smooth = np.arange(data_x[0], data_x[-1], CONFIG_INI.getfloat('smooth_model', 'resolution'))

We can now calculate the model on this smooth grid.

In [ ]:
# First dump all model parameters into tuple to access them easier
# (there is an easier way for this which we will implement at a later point)
params = (model.rl.val, model.flux0.val, model.epoch.val, model.inclin.val, model.MsMpR.val,
          model.ecc.val, model.omega.val, model.period.val, model.tzero.val, model.c1.val,
          model.c2.val, model.c3.val, model.c4.val, model.m_fac.val, model.hstp1.val,
          model.hstp2.val, model.hstp3.val, model.hstp4.val, model.xshift1.val,
          model.xshift2.val, model.xshift3.val, model.xshift4.val)

# Calculate model on denser grid to display
y_smooth_model = model.calc(pars=params, x=x_smooth)

In [ ]:
# And display that
plt.plot(x_smooth, y_smooth_model, c='orange')
plt.errorbar(data_x, data_y, yerr=uncertainty, fmt='.')
plt.xlabel('Phase')
plt.ylabel('Flux')
plt.title('Smooth model over simple data, before fit')

We can clearly see how the current model parameters are not a good fit to the data; especially the transit depth is way off. In order to make this better, we will now chose our statistic and an optimizer, and perform a fit.

Same as in the marginalisatino script, we will chose the chi-squared statistic and an LM optimizer.

In [ ]:
from sherpa.optmethods import LevMar
from sherpa.stats import Chi2

# Set up the statistic and optimizer
stat = Chi2()
opt = LevMar()
opt.config['epsfcn'] = np.finfo(float).eps   # adjusting epsfcn to double precision

print(stat)
print(opt)

So far, we have defined a data object and put that into a model object. We have settled on a statistic and an optimizer, so now we will combine the statistic and optimizer with the data and model to instantiate a fit object.

In [ ]:
from sherpa.fit import Fit

tfit = Fit(data, model, stat=stat, method=opt)
print(tfit)

What is left to do is to perform the fit and check the results.

In [ ]:
fitresult = tfit.fit()
print(fitresult)

We can access the covariance matrix of the fit directly, from where we calculate the parameter errors.

In [ ]:
hessian = np.sqrt(fitresult.extra_output['covar'].diagonal())
rl_err = hessian[0]

print('rl = {} +/- {}'.format(model.rl.val, rl_err))
print('Reduced chi-squared: {}'.format(fitresult.rstat))

And if we replot the new fit, we can see how this looks way better now.

In [ ]:
# Dump parameters in a single tuple again
params_result = (model.rl.val, model.flux0.val, model.epoch.val, model.inclin.val, model.MsMpR.val,
                 model.ecc.val, model.omega.val, model.period.val, model.tzero.val, model.c1.val,
                 model.c2.val, model.c3.val, model.c4.val, model.m_fac.val, model.hstp1.val,
                 model.hstp2.val, model.hstp3.val, model.hstp4.val, model.xshift1.val,
                 model.xshift2.val, model.xshift3.val, model.xshift4.val)

# Recalculate model on smooth grid now that we performed the fit
y_smooth_fit = model.calc(pars=params_result, x=x_smooth)

In [ ]:
# Plot the fit
plt.plot(x_smooth, y_smooth_fit, c='orange')
plt.errorbar(data_x, data_y, yerr=uncertainty, fmt='.')
plt.xlabel('Phase')
plt.ylabel('Flux')
plt.title('Smooth model over simple data, after fit')

## Limb darkening

The limb darkening coefficients used for the fitting get calculated in the main marginalisation script, and here we will briefly look into how this is done.

First, we usually read the limb darkening parameters of the stellar system in question from the configfile. In this tutorial, we set some of them manually.

In [ ]:
# Which stellar and planetary system are we working on?
exoplanet = 'simple_transit'
print('System: {}'.format(exoplanet))

# Read its limb darkening parameters
M_H = CONFIG_INI.getfloat(exoplanet, 'metallicity')    # stellar metallicity - limited ranges available
Teff = CONFIG_INI.getfloat(exoplanet, 'Teff')   # stellar temperature - for 1D models: steps of 250 starting at 3500 and ending at 6500
logg = CONFIG_INI.getfloat(exoplanet, 'logg')   # log(g), stellar gravity - depends on whether 1D or 3D limb darkening models are used
print('M_H: {}'.format(M_H))
print('Teff: {}'.format(Teff))
print('logg: {}'.format(logg))

# Define limb darkening directory, which is inside this package
# and read limb darkening model choice and grating
limbDir = os.path.join('..', 'Limb-darkening')
ld_model = '1D'
grat = 'G141'
print('LD model: {}'.format(ld_model))
print('Grating: {}'.format(grat))

We can now proceed to calculate the limb darkening coefficients. To do that, we need to load the wavelength array, wchich we will do for W17, on whichever grid is currently picked in the configfile.  
In the main script, the four non-linear limb darkening coefficients get used: c1, c2, c3 and c4 

In [ ]:
from limb_darkening import limb_dark_fit

# Load wavelength array
dataDir = os.path.join(CONFIG_INI.get('data_paths', 'input_path'), 'W17')
get_wvln = CONFIG_INI.get('W17', 'wvln_file')
wavelength = np.loadtxt(os.path.join(dataDir, get_wvln), skiprows=3)

_uLD, c1, c2, c3, c4, _cp1, _cp2, _cp3, _cp4, _aLD, _bLD = limb_dark_fit(grat, wavelength, M_H, Teff,
                                                                         logg, limbDir, ld_model)

print('\nc1 = {}'.format(c1))
print('c2 = {}'.format(c2))
print('c3 = {}'.format(c3))
print('c4 = {}'.format(c4))

## Full marginalisation

To launch the full marginalization over all systematic models, **run `marginalization.py`**. This script first performs a fit on all systematic models to scale the uncertainties to unity chi-squared, then it fits all systematic models again and saves the fit parameters for each of them. In a final step, the marginalization over all stsyematic models is performed, plotted and saved to `[data_paths] -> output_path`.

The reson we fit all models twice is the error estimation. The input flux errors are pure photon noise, which does not incorporate all information on real noise sources. The first fit is intended to understant additional noise sources. This is done by rescaling all uncertainties so that the data and the model have a reduced chi-squared of one. This means that the uncertainties will get slightly larger than pur photon noise and we use these more conservative (and probably more accurate) errors when running the second round of fitting, yielding fit parameters that we can trust better.